# 大作业1 业务报销单自动处理 学习笔记

视频教程地址：https://www.bilibili.com/video/BV1m7411j7Zm?spm_id_from=333.999.0.0&vd_source=aa068c732e0224662feb747f8c25f344

sendgrid 申请接口：https://justcode.ikeepstudying.com/2018/02/linux-shell%EF%BC%9A%E4%BD%BF%E7%94%A8-sendgrid-email-api-%E5%8F%91%E9%80%81%E9%82%AE%E4%BB%B6-sendgrid-app-google-cloud-platform-php-phython-shell-script-%E5%8F%91%E9%80%81%E9%82%AE%E4%BB%B6/

费用报销单模板下载：

## 代码

使用 sendgrid

In [ ]:
import pandas as pd
import smtplib
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders
from pathlib import Path


def read_checks(path, header=None):
    """
    read excel from file path
    
    param:
    path: pathlib.Path obj, pointing to file path
    
    return:
    row: dict, summary of the excel file
    """
    df = pd.read_excel(path, header=None)
    total = df.loc[df[1] == '总计']

    row = {}
    row['ID:'] = df.iloc[4, 10]
    row['姓名'] = df.iloc[3, 2]
    row['开始时间'] = df.iloc[7, 1]
    row['结束时间'] = df.iloc[8, 1]
    row['酒店']= total.iloc[0, 4]
    row['交通费']= total.iloc[0, 5]
    row['油费']= total.iloc[0, 6]
    row['餐饮']= total.iloc[0, 7]
    row['电话费']= total.iloc[0, 8]
    row['招待费']= total.iloc[0, 9]
    row['杂项']= total.iloc[0, 10]
    row['总计']= total.iloc[0, 11]
    
    return row


def send_mail(send_from, send_to, subject, text, file_path, server, port, username='', password='', tls=True):
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Date'] = formatdate(localtime= True)
    msg['Subject'] = subject
    msg.attach(MIMEText(text))
    
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(open(file_path, 'rb').read())
    encoders.encode_base64(part)
    part.add_header('Content-DIsposition', f'attachmen; filename="{file_path.name}"')  # use pathlib
    msg.attach(part)
    
    smtp = smtplib.SMTP(server, port)
    if tls:
        smtp.starttls()
        
    smtp.login(username, password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.quit()
    

def main():
    CWD = r'your working path, e.g. \pilot-student\assets\expenses'
    p = Path(CWD)
    checks = []
    send_from = 'mail address'
    send_to = 'mail address'
    subject = '报销数据汇总'
    text = '您好！报销数据已发送至您邮箱附件，请查收！祝好！'
    file_path = p / 'all.xlsx'
    server = 'smtp.sendgrid.net'
    port = '587'
    username = 'username'
    password = 'passwd'
    
    for q in p.glob('**/*.xlsx'):
        checks.append(read_checks(q))
        
    df = pd.DataFrame(checks)
    df.to_excel(p / 'all.xlsx')  # 用这个接口更方便些
    
    send_mail(send_from, send_to, subject, text, file_path, server, port, username=username, password=password)

使用 163 邮箱需要注意的地方

1. 163邮箱不支持tls加密，所以调用邮件发送函数时要写上 `tls=False`。
2. `smtp = smtplib.SMTP(server, port)` 这一句要改为 `smtp = smtplib.SMTP_SSL(server, port)`，否则会报错：
> smtplib.SMTPServerDisconnected: Connection unexpectedly closed

3. 账号为邮箱账号，密码需要申请独立授权码。

In [ ]:
import pandas as pd
import smtplib
import ssl
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders
from pathlib import Path


def read_checks(path, header=None):
    """
    read excel from file path
    
    param:
    path: pathlib.Path obj, pointing to file path
    
    return:
    row: dict, summary of the excel file
    """
    df = pd.read_excel(path, header=None)
    total = df.loc[df[1] == '总计']

    row = {}
    row['ID:'] = df.iloc[4, 10]
    row['姓名'] = df.iloc[3, 2]
    row['开始时间'] = df.iloc[7, 1]
    row['结束时间'] = df.iloc[8, 1]
    row['酒店']= total.iloc[0, 4]
    row['交通费']= total.iloc[0, 5]
    row['油费']= total.iloc[0, 6]
    row['餐饮']= total.iloc[0, 7]
    row['电话费']= total.iloc[0, 8]
    row['招待费']= total.iloc[0, 9]
    row['杂项']= total.iloc[0, 10]
    row['总计']= total.iloc[0, 11]
    
    return row


def send_mail(send_from, send_to, subject, text, file_path, server, port, username='', password='', tls=True):
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Date'] = formatdate(localtime= True)
    msg['Subject'] = subject
    msg.attach(MIMEText(text))
    
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(open(file_path, 'rb').read())
    encoders.encode_base64(part)
    part.add_header('Content-DIsposition', f'attachmen; filename="{file_path.name}"')  # use pathlib
    msg.attach(part)
    
    #smtp = smtplib.SMTP(server, port)
    smtp = smtplib.SMTP_SSL(server, port)
    if tls:
        smtp.starttls()
        
    smtp.login(username, password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.quit()
    

def main():
    CWD = r'your working path, e.g. \pilot-student\assets\expenses'
    p = Path(CWD)
    checks = []
    send_from = 'mail address'
    send_to = 'mail address'
    subject = '报销数据汇总'
    text = '您好！报销数据已发送至您邮箱附件，请查收！祝好！'
    file_path = p / 'all.xlsx'
    server = 'smtp.163.com'
    port = '994'
    username = '163 mail address'
    password = 'auth passwd'
    
    for q in p.glob('**/*.xlsx'):
        checks.append(read_checks(q))
        
    df = pd.DataFrame(checks)
    df.to_excel(p / 'all.xlsx')  # 用这个接口更方便些
    
    send_mail(send_from, send_to, subject, text, file_path, server, port, username=username, password=password, tls=False)